In [1]:
import tensorflow as tf
import numpy as np


In [38]:
def pretty_print_guess_tensor(const_guess, operand_guess, operator_guess):
    # TODO: const_guess

    # tf.debugging.assert_rank(const_guess, 3)
    tf.debugging.assert_rank(operand_guess, 2)
    tf.debugging.assert_rank(operator_guess, 3)

    s = []

    for t in operand_guess:
        s += [f'x_{tf.argmax(t)}']

    operator_lookup = ['+','-', '*','/']
    result = s[::]
    for i, op_one_hot in enumerate(operator_guess):
        operators = tf.argmax(op_one_hot,axis=-1)
        left = result[::2]
        right = (result[1:] + result[:1])[::2]
        ops = operators[:len(left)]
        result = []
        for l, op, r in zip(left, ops, right):
            result += [f'({l} {operator_lookup[op]} {r})']


    return ' '.join(result)

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)

pretty_print_guess_tensor(const_guess, operand_guess, operator_guess)

'(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))'

In [21]:
@tf.function
def to_prob_dist_all(v):
    v2 = tf.sqrt(tf.square(v)+1e-9)
    # v2 = tf.sqrt(tf.square(v))
    m = tf.expand_dims(tf.reduce_sum(v2, axis=-1),-1)
    n = tf.math.divide_no_nan(v2, m)
    return n

# tf.print(tf.argmax(operator_guess))
# tf.print(tf.argmax(to_prob_dist_all(operator_guess)))
tf.round(to_prob_dist_all([
[[1., 0, 0, 0], [1., 1, 0, 0]],
[[1., 1, 1, 0], [50, 25, 25, 0]]
])*100)

<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[100.,   0.,   0.,   0.],
        [ 50.,  50.,   0.,   0.]],

       [[ 33.,  33.,  33.,   0.],
        [ 50.,  25.,  25.,   0.]]], dtype=float32)>

In [22]:
@tf.function
def cross_entropy(x, y, epsilon = 1e-9):
    return -2 * tf.reduce_mean(y * tf.math.log(x + epsilon), -1) / tf.math.log(2.)

x = tf.constant([[
    [1.0,0],
    [0.5,0.5],
    [.75,.25]
    ]]
,dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch(x)
    y = cross_entropy(x, x)

tf.print(y)
tf.print(tape.gradient(y, x))

[[-0 1 0.811278105]]
[[[-1.44269502 29.8973541]
  [-0.442695022 -0.442695022]
  [-1.02765751 0.557305]]]


In [5]:
@tf.function
def entropy(x):
    return cross_entropy(x, x)

In [23]:
@tf.function
def dot(x, y):
    r = tf.multiply(x, y)
    return tf.reduce_sum(r, -1)

x = tf.constant([
    [[2,2,2]],
    [[3,3,3]]
])

dot(x, x)

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[12],
       [27]])>

In [85]:
@tf.function
def resolve_values(const_guess, values, operand_guess):
    # TODO: const_guess

    # tf.debugging.assert_rank(const_guess, 3)
    tf.debugging.assert_rank(values, 3) # [outer_batch, inner_batch, VALUES_SIZE]
    tf.debugging.assert_rank(operand_guess, 3) # [outer_batch, LEAVES_SIZE, VALUES_SIZE]

    values_shape = tf.shape(values)
    operands_shape = tf.shape(operand_guess)

    outer_batch, inner_batch, VALUES_SIZE = [values_shape[0], values_shape[1], values_shape[2]]
    outer_batch, LEAVES_SIZE, VALUES_SIZE = [operands_shape[0], operands_shape[1], operands_shape[2]]

    # Broadcast the operand choices
    operand_guess = tf.expand_dims(operand_guess, axis=1)
    operand_guess = tf.tile(operand_guess, [1,inner_batch,1,1]) # [outer_batch, inner_batch, LEAVES_SIZE, VALUES_SIZE]

    # Broadcast the values
    values = tf.expand_dims(values, axis=2)
    values = tf.tile(values, [1, 1, LEAVES_SIZE, 1]) # [outer_batch, inner_batch, LEAVES_SIZE, VALUES_SIZE]

    # Dot product
    operand_guess = to_prob_dist_all(operand_guess)
    result = dot(values, operand_guess) # [outer_batch, inner_batch, LEAVES_SIZE]

    return result

# v1 = tf.range(NUM_LEAVES)
# cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
# const_guess = tf.concat([cgv, cgv],axis=1)

operand_guess = tf.Variable([
    [[1,0,0], [0,1,0], [0,0,1]],
    [[1,1,0], [0,0,1], [1,1,1]],
],dtype=tf.float32)

values = tf.constant([
    [[1,2,3],
    [4,5,6]],
    [[7,8,9],
    [11,22,33]],
], dtype=tf.float32)

with tf.GradientTape() as tape:
    result = resolve_values(const_guess, values, operand_guess)

grads = tape.gradient(result, operand_guess)

tf.print(tf.round(result))
tf.print(grads)

[[[1 2 3]
  [4 5 6]]

 [[8 9 8]
  [17 33 22]]]
[[[-0.000190019608 0 0]
  [-0 -1.1920929e-07 0]
  [-0 -0 0.000190496445]]

 [[-3.00009489 2.99981 0]
  [-0 -0 0.00113773346]
  [-3.99999976 4.76837158e-07 4]]]


In [136]:
# operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
operand_guess = to_prob_dist_all(tf.constant([
    [[ 1,1,1,1, 1,1,1,1]],
    [[ 1,1,1,1, 1,1,1,1]],
],dtype=tf.float32))

operand_guess = tf.Variable(operand_guess)
values = tf.constant([
    [[1,2,3,4, 5,6,7,8],
    [2,4,6,8,10,12,14,16]],
    [[1,2,3,4, 5,6,7,8],
    [3,6,9,12,15,18,21,24]],
],dtype=tf.float32)
target = tf.constant([
    [[3],[6]],
    [[3],[9]]
],dtype=tf.float32)
with tf.GradientTape() as tape:
    resolved = resolve_values(const_guess, values, operand_guess)
    loss = tf.nn.l2_loss(resolved - target)

grads = tape.gradient(loss, operand_guess)

# tf.print(tf.round(resolved))
# # tf.print(resolved - target)
# print(grads)
tf.print(operand_guess)
tf.print(entropy(operand_guess))

[[[0.125 0.125 0.125 ... 0.125 0.125 0.125]]

 [[0.125 0.125 0.125 ... 0.125 0.125 0.125]]]
[[0.75]
 [0.75]]


In [141]:
# operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
operand_guess = tf.constant([
    [[ 1,1,1,1, 1,1,1,1]],
    [[ 1,1,1,1, 1,1,1,1]],
],dtype=tf.float32)

operand_guess = tf.Variable(operand_guess)
values = tf.constant([
    [[1,2,3,4, 5,6,7,8],
    [2,4,6,8,10,12,14,16]],
    [[1,2,3,4, 5,6,7,8],
    [3,6,9,12,15,18,21,24]],
],dtype=tf.float32)
target = tf.constant([
    [[3],[6]],
    [[3],[9]]
],dtype=tf.float32)

# opt = tf.keras.optimizers.Adam(3e-4)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-1,
    decay_steps=100,
    decay_rate=1e-1,
    staircase=True)

opt = tf.keras.optimizers.Adam(lr_schedule)

tf.print('grad_mag\tloss\ttarget_loss\tentropy_loss\tresolved\tdist')
steps = 1000
for i in range(steps):
    with tf.GradientTape() as tape:
        resolved = resolve_values(const_guess, values, operand_guess)

        target_loss = tf.nn.l2_loss(resolved - target)
        entropy_loss = tf.reduce_sum(entropy(operand_guess))

        loss = target_loss + entropy_loss * 1e+1
    
    variables = [operand_guess]
    grads = tape.gradient(loss, variables)
    grad_mag = tf.sqrt(tf.reduce_sum(tf.square(grads)))
    opt.apply_gradients(zip(grads, variables))

    operand_guess.assign(to_prob_dist_all(operand_guess))

    if i % (steps // 10) == 0:
        print_idx = 0
        dist = list(tf.round(operand_guess * 100).numpy().astype(np.int32))
        # tf.print(tf.squeeze(resolved)[print_idx], loss[print_idx], target_loss, entropy_loss[print_idx], dist[print_idx])
        tf.print(grad_mag, loss, target_loss, entropy_loss, tf.squeeze(resolved)[print_idx],  dist[print_idx])

np.vstack(tf.round(operand_guess * 100).numpy().astype(np.int32))

grad_mag	loss	target_loss	entropy_loss	resolved	dist
19.8168411 16.875 16.875 -0 [4.5 9] array([[13, 13, 13, 13, 13, 13, 12, 12]])
39.5153236 9.73279381 1.30473399 0.842806 [3.65213013 7.30426025] array([[ 0,  6, 74,  4,  4,  4,  4,  4]])
66.4609299 3.12039328 0.0624751598 0.305791795 [3.07909274 6.15818548] array([[ 0,  0, 97,  1,  1,  1,  0,  0]])
95.6510544 1.79596531 0.0517166853 0.174424857 [3.00281906 6.00563812] array([[  0,   0, 100,   0,   0,   0,   0,   0]])
109.240227 1.6084348 0.0509988107 0.155743599 [3.00089931 6.00179863] array([[  0,   0, 100,   0,   0,   0,   0,   0]])
115.199 1.59715235 0.0508324131 0.154632 [3.00067759 6.00135517] array([[  0,   0, 100,   0,   0,   0,   0,   0]])
92.7105637 1.67991674 0.0490814187 0.163083524 [3.00306964 6.00613928] array([[  0,   0, 100,   0,   0,   0,   0,   0]])
86.9043427 1.73169053 0.0478962697 0.168379426 [3.00478315 6.00956631] array([[  0,   0, 100,   0,   0,   0,   0,   0]])
83.7797089 1.76938772 0.0470183194 0.172236949 [3.

array([[  0,   0, 100,   0,   0,   0,   0,   0],
       [  0,  85,   0,   0,   0,   0,   0,  15]])

In [8]:
operands = tf.range(NUM_LEAVES, dtype=tf.float32)
v2 = tf.range(NUM_OPERATORS)
operators = tf.constant([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1],
],dtype=tf.float32)
# operators = tf.constant([
#     [1,0,0,0],
#     [1,0,0,0],
#     [1,0,0,0],
#     [1,0,0,0],
# ],dtype=tf.float32)

operands = tf.Variable(operands)
operators = tf.Variable(operators)

# opt = tf.keras.optimizers.Adam(3e-4)
opt = tf.keras.optimizers.Adam(1e-2)
# opt = tf.keras.optimizers.SGD(1e-2)

target = tf.constant([1,5,9,13],dtype=tf.float32)

@tf.function
def operate(operands, operators):
    left = operands[::2]
    right = tf.roll(operands, shift=-1, axis=0)[::2]

    r_add = left + right
    r_sub = left - right
    r_mul = left * right
    r_div = tf.math.divide_no_nan(left, right)

    r = tf.stack([r_add, r_sub, r_mul, r_div], axis=1)

    # operators = tf.nn.softmax(operators, axis=-1)
    operators = to_prob_dist_all(operators)

    return dot(r, operators)



152.184158 -0 [0.999873519 -0.999536216 19.9983807 0.858769] [0 1 2 3]
0.0415764526 2.35339117 [0.984164834 5.28512764 9.03927135 12.9921169] [0 2 2 3]
0.0340516455 2.36072421 [0.981587708 5.25888062 9.01051903 12.9748116] [0 2 2 3]
0.041841168 2.32249355 [0.980450511 5.28663635 9.01120377 12.968153] [0 0 2 3]
0.086477 2.19131327 [0.978189945 5.41216087 9.01288605 12.9506474] [0 0 2 3]
0.0639940351 1.94902551 [0.976990163 5.35166645 9.01619434 12.9406109] [0 0 2 3]
0.0412553661 1.79181194 [0.9779616 5.27859306 9.01608849 12.9355631] [0 0 2 3]
0.0186342839 1.72558236 [0.97842139 5.17910862 9.0161171 12.9331923] [0 0 2 3]
0.00273088086 1.50006711 [0.979037404 5.00619078 9.01615334 12.9312754] [0 0 2 3]
0.00394719327 1.36542809 [0.979485512 4.95314 9.01621 12.929184] [0 0 2 3]


In [ ]:
@tf.function
def operate_train_step(operands, operators, target):
    with tf.GradientTape() as tape:
        result = operate(operands, operators)
        target_loss = tf.nn.l2_loss(target - result)

        entropy_loss = tf.reduce_sum(entropy(operators))

        loss = target_loss + entropy_loss

    variables = [operators]
    grads = tape.gradient(loss, variables)

    opt.apply_gradients(zip(grads, variables))
    operators.assign(to_prob_dist_all(operators))

    return loss, target_loss, entropy_loss, result

steps = 1000
for i in range(steps):
    loss, target_loss, entropy_loss, result = operate_train_step(operands, operators, target)

    if i % (steps // 10) == 0:
        tf.print(target_loss, entropy_loss, result, tf.argmax(operators, axis=-1))

In [9]:
def eager_process_block(operands, operators_arr):
    acc = operands

    for operators in operators_arr:
        num_operands = tf.shape(acc)[0]
        operators = operators[:num_operands // 2]
        acc = operate(acc, operators)

    return acc

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)
operands = resolve_values(const_guess, values, operand_guess)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operator_guess)
    result = eager_process_block(operands, operator_guess)

tf.print(pretty_print_guess_tensor(const_guess, operand_guess, operator_guess))
x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7 = list(range(8))
tf.print((((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7))))
tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operator_guess))

(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
19.142857142857142
[19.1372566]
[[1.00041687 1.00029039 1.00045919 -1.00023425]
 [4.99858665 3.99898624 -0.143142194 0.122262307]]
[[[0.000126719475 0 0 0]
  [0 -0.000464081764 0 0]
  [0 0 0.00161933899 0]
  [0 0 0 0.00162553787]]

 [[2.67056748e-07 0 0 0]
  [0 -0.000123977661 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0.00241851807 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [10]:
@tf.function
def unrolled_process_block_3(operands, operators_arr):
    acc = operands

    # Level 1
    operators = operators_arr[0]
    operators = operators[:4]
    acc = operate(acc, operators)

    # Level 2
    operators = operators_arr[1]
    operators = operators[:2]
    acc = operate(acc, operators)

    # Level 3
    operators = operators_arr[2]
    operators = operators[:1]
    acc = operate(acc, operators)

    return acc

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)
operands = resolve_values(const_guess, values, operand_guess)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operator_guess)
    result = unrolled_process_block_3(operands, operator_guess)

tf.print(pretty_print_guess_tensor(const_guess, operand_guess, operator_guess))
x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7 = list(range(8))
tf.print((((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7))))
tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operator_guess))

(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
19.142857142857142
[19.1372566]
[[1.00041687 1.00029039 1.00045919 -1.00023425]
 [4.99858665 3.99898624 -0.143142194 0.122262307]]
[[[0.000126719475 0 0 0]
  [0 -0.000464081764 0 0]
  [0 0 0.00161933899 0]
  [0 0 0 0.00162553787]]

 [[2.67056748e-07 0 0 0]
  [0 -0.000123977661 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0.00241851807 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [11]:
def print_collapsed_result(values, const_guess, operand_guess, operator_guess):
    # TODO: const_guess

    operands = tf.round(operand_guess)
    acc = resolve_values(const_guess, values, operands)
    operators = tf.round(operator_guess)

    result = eager_process_block(acc, operators)

    return result

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)

print_collapsed_result(values, const_guess, operand_guess, operator_guess)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([19.137257], dtype=float32)>

In [12]:
def bind_opt_train_step(opt, entropy_weight=1e+2):
    @tf.function
    def train_step(const_guess, operand_guess, operator_guess, values, target):
        with tf.GradientTape() as tape:
            cg, opg, otg = const_guess, operand_guess, operator_guess
            # cg = tf.nn.softmax(cg)
            # opg = tf.nn.softmax(opg)
            # otg = tf.nn.softmax(otg)

            cg = to_prob_dist_all(cg)
            opg = to_prob_dist_all(opg)
            otg = to_prob_dist_all(otg)

            cg_entropy = 0.0 # TODO
            opg_entropy = tf.reduce_sum(entropy(opg))
            otg_entropy = tf.reduce_sum(entropy(otg))

            operands = resolve_values(cg, values, opg)
            result = unrolled_process_block_3(operands, otg)

            target_loss = tf.nn.l2_loss(result[0] - target)

            loss = target_loss

            if target_loss < 1:
                loss += entropy_weight * (opg_entropy + otg_entropy)

        variables = [operand_guess, operator_guess]
        grads = tape.gradient(loss, variables)
        # grads = [tf.clip_by_norm(g, 100.0) for g in grads]
        opt.apply_gradients(zip(grads, variables))

        const_guess.assign(to_prob_dist_all(const_guess))
        operand_guess.assign(to_prob_dist_all(operand_guess))
        operator_guess.assign(to_prob_dist_all(operator_guess))

        return loss, target_loss, cg_entropy, opg_entropy, otg_entropy

    return train_step

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)

const_guess = tf.Variable(const_guess)
operand_guess = tf.Variable(operand_guess)
operator_guess = tf.Variable(operator_guess)

target = 19.0

opt = tf.keras.optimizers.Adam(3e-4)
# opt = tf.keras.optimizers.SGD(1e-1)
train_step = bind_opt_train_step(opt)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(const_guess)
    tape.watch(operand_guess)
    tape.watch(operator_guess)

    result = train_step(const_guess, operand_guess, operator_guess, values, target)

tf.print(result)
tf.print(tape.gradient(result, operand_guess))
tf.print(tape.gradient(result, operator_guess))

(1.66757882, 0.00910403579, 0, 0.00725521892, 0.00932952948)
[[-0.0836830139 0 0 ... 0 0 0]
 [0 -0.0836334229 0 ... 0 0 0]
 [0 0 -0.0835838318 ... 0 0 0]
 ...
 [0 0 0 ... -0.0832290649 0 0]
 [0 0 0 ... 0 -0.083530426 0]
 [0 0 0 ... 0 0 -0.0834960938]]
[[[-0.0715789795 0 0 0]
  [0 -0.0716934204 0 0]
  [0 0 -0.0712966919 0]
  [0 0 0 -0.0712966919]]

 [[-0.0716018677 0 0 0]
  [0 -0.0716323853 0 0]
  [0 0 -0.0716095 0]
  [0 0 0 -0.0716095]]

 [[-0.071144104 0 0 0]
  [0 -0.0716095 0 0]
  [0 0 -0.0716095 0]
  [0 0 0 -0.0716095]]]


In [13]:
# opt = tf.keras.optimizers.Adam(3e-4)
opt = tf.keras.optimizers.Adam(1e-2)
# opt = tf.keras.optimizers.SGD(1e-1)
train_step = bind_opt_train_step(opt, 1)

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)

const_guess = tf.Variable(const_guess)
operand_guess = tf.Variable(operand_guess)
operator_guess = tf.Variable(operator_guess)

target = 7.0
steps = 1000
for i in range(steps):
    loss, target_loss, cg_entropy, opg_entropy, otg_entropy = train_step(const_guess, operand_guess, operator_guess, values, target)

    if i % (steps // 10) == 0:
        cg = const_guess.numpy()
        opg = operand_guess.numpy()
        otg = operator_guess.numpy()
        collapsed_result = print_collapsed_result(values, cg, opg, otg)

        tf.print(i, collapsed_result[0], loss, target_loss, cg_entropy, opg_entropy, otg_entropy)
        # tf.print(pretty_print_guess_tensor(cg, opg, otg))
tf.print(pretty_print_guess_tensor(cg, opg, otg))

0 19.1372566 73.6283493 73.6283493 0 0.00725521892 0.00932952948
100 19.1372566 2.81111312 0.00345588336 0 1.18885708 1.61880016
200 19.1372566 2.19650316 0.00878225453 0 0.898448706 1.28927231
300 19.1372566 1.9347347 7.87916069e-05 0 0.862380922 1.07227492
400 19.1372566 1.96360183 0.104608402 0 0.793349147 1.06564426
500 19.1372566 1.87586272 0.00224292884 0 0.894468546 0.979151189
600 19.1372566 1.78837192 0.043634709 0 0.872879863 0.871857405
700 19.1372566 1.82108438 0.0674607903 0 0.917027 0.836596608
800 19.1372566 2.20998025 0.530530274 0 0.905871391 0.773578525
900 19.1372566 2.08362675 0.372966111 0 0.917841613 0.792819
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))


In [14]:
tf.round(opg * 100)


<tf.Tensor: shape=(8, 8), dtype=float32, numpy=
array([[98.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0., 98.,  1.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0., 98.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0., 98.,  0.,  1.,  0.,  0.],
       [23.,  0.,  0.,  0., 76.,  0.,  0.,  0.],
       [33., 10.,  0.,  0.,  1., 56.,  0.,  0.],
       [ 1.,  1.,  0.,  1.,  0.,  0., 97.,  1.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  1., 96.]], dtype=float32)>

In [15]:
tf.round(entropy(opg)*100)

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([ 5.,  5.,  4.,  6., 22., 35.,  7.,  7.], dtype=float32)>

In [84]:
@tf.function
def resolve_values(const_guess, values, operand_guess):
    # TODO: const_guess

    # tf.debugging.assert_rank(const_guess, 3)
    tf.debugging.assert_rank(values, 3) # [outer_batch, inner_batch, VALUES_SIZE]
    tf.debugging.assert_rank(operand_guess, 3) # [outer_batch, LEAVES_SIZE, VALUES_SIZE]

    values_shape = tf.shape(values)
    operands_shape = tf.shape(operand_guess)

    outer_batch, inner_batch, VALUES_SIZE = [values_shape[0], values_shape[1], values_shape[2]]
    outer_batch, LEAVES_SIZE, VALUES_SIZE = [operands_shape[0], operands_shape[1], operands_shape[2]]

    # Broadcast the operand choices
    operand_guess = tf.expand_dims(operand_guess, axis=1)
    operand_guess = tf.tile(operand_guess, [1,inner_batch,1,1]) # [outer_batch, inner_batch, LEAVES_SIZE, VALUES_SIZE]

    # Broadcast the values
    values = tf.expand_dims(values, axis=2)
    values = tf.tile(values, [1, 1, LEAVES_SIZE, 1]) # [outer_batch, inner_batch, LEAVES_SIZE, VALUES_SIZE]

    # Dot product
    operand_guess = to_prob_dist_all(operand_guess)
    result = dot(values, operand_guess) # [outer_batch, inner_batch, LEAVES_SIZE]

    return result

# v1 = tf.range(NUM_LEAVES)
# cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
# const_guess = tf.concat([cgv, cgv],axis=1)

operand_guess = tf.Variable([
    [[1,0,0], [0,1,0], [0,0,1]],
    [[1,1,0], [0,0,1], [1,1,1]],
],dtype=tf.float32)

values = tf.constant([
    [[1,2,3],
    [4,5,6]],
    [[7,8,9],
    [11,22,33]],
], dtype=tf.float32)

with tf.GradientTape() as tape:
    result = resolve_values(const_guess, values, operand_guess)

grads = tape.gradient(result, operand_guess)

tf.print(tf.round(result))
tf.print(grads)

[[[1 2 3]
  [4 5 6]]

 [[8 9 8]
  [17 33 22]]]
[[[-0.000190019608 0 0]
  [-0 -1.1920929e-07 0]
  [-0 -0 0.000190496445]]

 [[-3.00009489 2.99981 0]
  [-0 -0 0.00113773346]
  [-3.99999976 4.76837158e-07 4]]]
